# Expences analysis for Tinkoff  debit card.

1. Download all your transactions data at https://www.tinkoff.ru/events/feed/ (press `...`) and save it to `"./operations.xls"` file
2. Run the code

## TODO
1. Plots in time by category
2. Consider income transactions - outcome
2. Descriptions

In [ ]:
import pandas as pd
import seaborn as sns

from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
import matplotlib.pyplot as plt
output_notebook()

In [ ]:
tinkoff_datapath = "./operations.xls"
tinkoff_data = pd.read_excel(tinkoff_datapath, encoding="ascii")

In [ ]:
tinkoff_data.head()

In [ ]:
tinkoff_data.columns = [
    "time", "date_op", "card", "status", 
    "amount_op", "currency_op", "amount", "currency",
    "cashback", "category", "MCC", "description", "bonuses"]
tinkoff_data = tinkoff_data[tinkoff_data.status == "OK"]
tinkoff_data.drop([
    "date_op", "card", "status", "cashback", "bonuses"
], axis=1, inplace=True)
tinkoff_data.time = pd.to_datetime(tinkoff_data.time, format="%d.%m.%Y %H:%M:%S")
period = tinkoff_data.time.max() - tinkoff_data.time.min()

In [ ]:
monthes = period.days/30.4

In [ ]:
tinkoff_data.describe()

In [ ]:
tinkoff_data.describe(exclude=["float64"])

In [ ]:
tinkoff_data.currency.unique()

In [ ]:
rub_expenses = tinkoff_data[(tinkoff_data.currency_op == "RUB") & 
                            (tinkoff_data.amount_op < 0)]


In [ ]:
rub_expenses

In [ ]:
data = -rub_expenses.groupby("category").agg({"amount_op": "sum"})/monthes
data.sort_values("amount_op", inplace=True)
data = data[data.amount_op > 50]
data = data[~data.index.isin(["Финан. услуги", "Переводы", "Переводы/иб", "Финан. услуги"])]
print("Mounthly expenses:", data.sum())
source = ColumnDataSource(data)
p = figure(y_range=data.index.tolist(), plot_height=550, plot_width=950, title="Expenses by category. Total")
p.hbar(y="category", right="amount_op", height=0.9, source=source)
p.x_range.start = 0
show(p)

In [ ]:
data.plot.pie(y='amount_op', figsize=(10, 10), legend=False)

In [ ]:
data = -rub_expenses[rub_expenses.category == "Супермаркеты"].groupby("description").agg({"amount_op": "sum"})/monthes
data.sort_values("amount_op", inplace=True)
#data = data[data.amount_op > 1000]
#data = data[~data.index.isin(["Финан. услуги", "Переводы", "Переводы/иб", "Наличные", "Финан. услуги"])]
source = ColumnDataSource(data)
p = figure(y_range=data.index.tolist(), plot_height=550, plot_width=950, title="Expenses by category. Total")
p.hbar(y="description", right="amount_op", height=0.9, source=source)
p.xgrid.grid_line_color = None
p.x_range.start = 0
show(p)

In [ ]:
data = -rub_expenses.groupby("description").agg({"amount_op": "sum"})/monthes
data.sort_values("amount_op", inplace=True)
data = data[data.amount_op > 500]
source = ColumnDataSource(data)
p = figure(y_range=data.index.tolist(), plot_height=1000, plot_width=950, title="Expenses by category. Total")
p.hbar(y="description", right="amount_op", height=0.9, source=source)
p.xgrid.grid_line_color = None
p.x_range.start = 0
show(p)